In [1]:
# Импорты
import pandas as pd
import gdown
import joblib
import numpy as np
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
from ml_utils import calculate_metrics
pd.options.mode.chained_assignment = None
random.seed(42)
np.random.seed(42)

In [ ]:
# Загрузка данных для обучения пайплайна

# y_hubs
# h_id = "1hMUvpXsJGHVlrUB-dRNkkGxzK5uZsE0i"
# gdown.download(f"https://drive.google.com/uc?id={h_id}", quiet=False)

# # y_rating
# r_id = "1kRKU24Ah8JC1RDokiyz22-6EUNYFPqB-"
# gdown.download(f"https://drive.google.com/uc?id={r_id}", quiet=False)

# # Матрица признаков X
# X_id = "1nyfcMkhKS0yVzGgZBJhQNiu572bJlN1j"
# gdown.download(f"https://drive.google.com/uc?id={X_id}", quiet=False)

In [2]:
# Данные
X = pd.read_parquet('X.parquet')['text_combined']
y_rating = pd.read_csv('y_rating.csv')['rating_level']
y_hubs = pd.read_csv("y_hubs.csv", header=None, dtype=int).values

# Лучшие параметры для пайплайнов
best_params_h = joblib.load('best_params/best_params_h.pkl')
best_params_r = joblib.load('best_params/best_params_r.pkl')

In [3]:
# Пайплайн с лучшими параметрами для предсказания хабов (тем) статей
pipeline_h = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LinearSVC(max_iter=1000), n_jobs=6))])

# Настройка пайплайна с лучшими параметрами
pipeline_h.set_params(**best_params_h)

# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_hubs, test_size=0.25)

# Обучение и предсказание по лучшему пайплайну для хабов
pipeline_h.fit(X_train, y_train_h)
y_pred_h = pipeline_h.predict(X_test)

In [4]:
# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_h, y_test_h, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.3818, 'Recall': 0.7199, 'F1-Score': 0.499, 'Hamming Loss': 0.0167}
macro - {'Precision': 0.374, 'Recall': 0.703, 'F1-Score': 0.4743, 'Hamming Loss': 0.0167}
weighted - {'Precision': 0.4382, 'Recall': 0.7199, 'F1-Score': 0.5345, 'Hamming Loss': 0.0167}


### Рейтинг

In [5]:
# Пайплайн с лучшими параметрами для предсказания рейтинга статей
pipeline_r = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag'), n_jobs=6))])

# Настройка пайплайна с лучшими параметрами
pipeline_r.set_params(**best_params_r)

# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
X_train, X_test, y_train_r, y_test_r = train_test_split(X, y_rating, test_size=0.25)

# Обучение и предсказание по лучшему пайплайну для рейтинга
pipeline_r.fit(X_train, y_train_r)
y_pred_r = pipeline_r.predict(X_test)

In [6]:
# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_r, y_test_r, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.4958, 'Recall': 0.4958, 'F1-Score': 0.4958, 'Hamming Loss': 0.5042}
macro - {'Precision': 0.2893, 'Recall': 0.4746, 'F1-Score': 0.2886, 'Hamming Loss': 0.5042}
weighted - {'Precision': 0.5601, 'Recall': 0.4958, 'F1-Score': 0.5174, 'Hamming Loss': 0.5042}
